# 3. Enable detecting changes of terminal operations in ports

This section is designed for learners who are interested in detecting changes of terminal operations in ports with ALOS-2 - case study of Nagoya, Japan.

## Here are the key learning objectives:
- Preprocess to reduce speckle noises
- Visual ALOS-2 satellite imagery for Toyota's facility in Nagoya port
- Change detection of the port operation in the COVID-19 crisis

## 3.1 Preprocess to reduce speckle noises

In this section, we focus on preparing ALOS-2 Synthetic Aperture Radar (SAR) imagery for analysis.
- downloading ALOS-2 images
- calibrated to adjust for sensor-related biases
- apply a Lee filter to reduce speckle noise

**Download ALOS-2 imagery from Owncloud, and then unzip for the next step: data processing**

In [ ]:
%cd /home/jovyan/shared/Sirinya/alos/ALOS2.1
!wget --content-disposition "https://owncloud.glodal-inc.net/owncloud/index.php/s/7LpA8rRJjIUFabL/download"

In [ ]:
!unzip "Shimpomachi_ngo.zip"

**Load the ALOS-2 images**

This Python code is for loading and reading ALOS-2 SAR images by loading the paths of all .tif files from the specified folder.

In [ ]:
pip install rasterio

In [ ]:
import os

# Define the path to the ALOS-2 data folder
data_folder = '/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/data'

# Find all GeoTIFF files in the folder
alos_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.tif')]
if not alos_files:
    raise FileNotFoundError("No GeoTIFF files found in the specified directory.")
    
# Display list of files found
print("ALOS-2 files to visualize:")
for file in alos_files:
    print(file)


**Visualize Images**

This cell will visualize each image found in the folder, without displaying metadata.

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import os

# Choose a color map
color_map = 'gray'  # Options: 'gray', 'viridis', 'plasma', 'inferno', 'jet', 'cividis', etc.

# Define the number of columns for the grid
n_cols = 2
n_rows = (len(alos_files) + n_cols - 1) // n_cols  # Calculate required rows based on the number of files

# Create a figure with subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 6))
axes = axes.flatten()  # Flatten to make it easy to iterate over

# Loop over each file and visualize the image with the selected color map
for idx, alos_path in enumerate(alos_files):
    with rasterio.open(alos_path) as src:
        alos_image = src.read(1)  # Read the first band (Assuming single-band data)

    # Plot the image in the respective subplot
    ax = axes[idx]
    img = ax.imshow(alos_image, cmap=color_map)
    ax.set_title(f"{os.path.basename(alos_path)} ({color_map})")
    ax.set_xlabel("X coordinate")
    ax.set_ylabel("Y coordinate")
    fig.colorbar(img, ax=ax)

# Hide any unused subplots
for j in range(idx + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


**Calibrate Images**

After obtaining the data, apply radiometric calibration factors to convert digital numbers (DN) to backscatter values (dB) using the below formular:

- σ0 = 10*log10(DN2) - 83.0 dB [ALOS-2 Calibration Result of ALOS-2](https://www.eorc.jaxa.jp/ALOS/en/alos-2/a2_calval_e.htm)

NOTE: Radiometric calibration is essential for translating ALOS-2 SAR data into meaningful backscatter values that reflect the Earth's surface accurately. This process corrects for sensor biases, making the data comparable across different sensors and acquisition times. Proper calibration enables detailed analyses, such as environmental monitoring, land use classification, and disaster management, by ensuring consistent, physically accurate measurements.

NOTE: The formula above is the same with the formula: 20 * np.log10(ALOS-2 data) - 83.0)

In [7]:
import numpy as np
import rasterio

# Define a function to calibrate the image
def calibrate_image(image):
    # Clip negative values to avoid taking log of non-positive values
    clipped_image = np.clip(image, a_min=1e-10, a_max=None)  # Prevent log(0)
    # Apply calibration formula
    calibrated_image = (20 * np.log10(clipped_image)) - 83
    return calibrated_image

# Apply calibration to original images and store in a dictionary
calibrated_images = {}
for alos_path in alos_files:
    with rasterio.open(alos_path) as src:
        alos_image = src.read(1)  # Read the first band (Assuming single-band data)

    # Calibrate the original image
    calibrated_images[alos_path] = calibrate_image(alos_image)


**Apply Lee Filter to Calibrated Images**

The Lee filter works by computing the local statistics (mean and variance) over a moving window and uses this to reduce speckle noise while preserving the edges. This cell defines the lee_filter function and applies it to each ALOS-2 image in the directory. It stores the filtered images in a dictionary, filtered_images.

Note: The size=3 parameter in the lee_filter() function refers to the size of the square window or kernel used to compute the local statistics (mean, variance) for each pixel. In this case, a 3x3 window is applied around each pil.

In [8]:
from scipy.ndimage import uniform_filter

# Define the Lee filter function
def lee_filter(image, size):
    mean = uniform_filter(image, size=size)
    mean_sqr = uniform_filter(image**2, size=size)
    variance_img = mean_sqr - mean**2
    overall_variance = variance_img.mean()
    weight = variance_img / (variance_img + overall_variance)
    filtered_image = mean + weight * (image - mean)
    return filtered_image

# Apply Lee filter to the calibrated images
filtered_images = {}
for alos_path, calibrated_image in calibrated_images.items():
    filtered_images[alos_path] = lee_filter(calibrated_image, size=3)  # Adjust size as needed


**Visualize Original, Calibrated, and Filtered Images**

In [ ]:
import matplotlib.pyplot as plt
import os
import rasterio

# Visualize Original, Calibrated, and Filtered Images side-by-side
for alos_path in alos_files:
    # Retrieve the original, calibrated, and filtered images
    with rasterio.open(alos_path) as src:
        alos_image = src.read(1)
    calibrated_image = calibrated_images[alos_path]
    filtered_image = filtered_images[alos_path]

    # Plot
    plt.figure(figsize=(15, 5))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(alos_image, cmap='gray')
    plt.title(f"Original Image - {os.path.basename(alos_path)}")
    plt.colorbar()

    # Calibrated Image
    plt.subplot(1, 3, 2)
    plt.imshow(calibrated_image, cmap='gray')
    plt.title(f"Calibrated Image - {os.path.basename(alos_path)}")
    plt.colorbar()

    # Filtered Image
    plt.subplot(1, 3, 3)
    plt.imshow(filtered_image, cmap='gray')
    plt.title(f"Filtered Image - {os.path.basename(alos_path)}")
    plt.colorbar()

    plt.tight_layout()
    plt.show()


**(Optional) Export the filtered images as GeoTIFF files**

In [ ]:
import rasterio

# Define the output path to save filtered GeoTIFF images
output_folder = '/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Export filtered images as GeoTIFF
for alos_path, filtered_image in filtered_images.items():
    output_filename = os.path.join(output_folder, os.path.basename(alos_path).replace('.tif', '_filtered.tif'))
    
    with rasterio.open(alos_path) as src:
        # Get metadata from the original image
        meta = src.meta.copy()
        meta.update({
            'height': filtered_image.shape[0],
            'width': filtered_image.shape[1],
            'dtype': 'float32',  # Change to 'float32' for filtered images
        })
    
    # Write the filtered image to a new GeoTIFF file
    with rasterio.open(output_filename, 'w', **meta) as dst:
        dst.write(filtered_image.astype('float32'), 1)  # Write the first band

    print(f"Filtered image saved as: {output_filename}")

## 3.2. Visual ALOS-2 satellite imagery for Toyota's facility in Nagoya port
Brief introduction to this section

- Visualize several observations around production suppression under COVID-19 crisis. Compare them with high-res image on Google Earth for sampled locations.
- Visualize the changes with a color composite and explain teh relevance with the production suppression.
- Show a time-series profile for a sampled location/region indicating the preduction suppression in TOYOTA.

### 3.2.1 Visualize several observations around production suppression under COVID-19 crisis. Compare them with high-res image on Google Earth for sampled locations.

**Visualizing the filtered images**

In [ ]:
import os
import rasterio
import matplotlib.pyplot as plt

# Define the path to the folder containing filtered images
filtered_images_folder = '/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img'

# Find all GeoTIFF files in the folder
filtered_files = [os.path.join(filtered_images_folder, f) for f in os.listdir(filtered_images_folder) if f.endswith('.tif')]
if not filtered_files:
    raise FileNotFoundError("No GeoTIFF files found in the specified directory.")

# Display list of filtered files found
print("Filtered ALOS-2 files to visualize:")
for file in filtered_files:
    print(file)

# Choose a color map
color_map = 'gray'  # You can change the color map as needed

# Determine number of rows needed for the subplots
num_images = len(filtered_files)
num_columns = 2
num_rows = (num_images + num_columns - 1) // num_columns  # Ceiling division to get rows

# Create a figure with subplots
plt.figure(figsize=(15, num_rows * 5))  # Adjust figure size based on number of rows

# Loop over each filtered file and add to the subplot
for i, filtered_path in enumerate(filtered_files):
    with rasterio.open(filtered_path) as src:
        filtered_image = src.read(1)  # Read the first band (Assuming single-band data)

    # Create a subplot for each image
    plt.subplot(num_rows, num_columns, i + 1)
    plt.imshow(filtered_image, cmap=color_map)
    plt.colorbar()
    plt.title(f"{os.path.basename(filtered_path)}")
    plt.xlabel("X coordinate")
    plt.ylabel("Y coordinate")

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


**Compare the observations with Sentinel-2 image for sampled locations**

By comparing observations from these two sources on matching dates, you can analyze how features or land cover in the sampled locations changed over time, gaining a fuller picture by using both SAR and optical perspectives. This can highlight subtle changes not visible in just one data type.

- ALOS-2 (SAR): These images provide insights into surface changes and structures, regardless of weather or lighting conditions. The three dates (191127, 210303, and 220302) allow you to track changes in infrastructure and surface roughness across pre-crisis, in-crisis, and post-crisis periods.
- Sentinel-2 (Optical): These images capture reflectance in visible and near-infrared bands. Comparing the Sentinel-2 images (November 2019, March 2021, March 2022) provides insights into land cover changes and activities around interesting areas.

|The observations (ALOS-2 image : 191127)| 
|-----------------------|
|![ALOS-2_1](9fil191127.png)|

|ALOS-2 image (filtered image)|Sentinel-2 image (Nov19)|
|------------------------|---------------------|
|A|
|![ALOS-2_A](9fil_1.png)|![Sentinel-2_A](9fil_2.png)|
|B|
|![ALOS-2_B](9fil_3.png)|![Sentinel-2_B](9fil_4.png)|
|C|
|![ALOS-2_C](9fil_5.png) |![Sentinel-2_C](9fil_6.png)|

|The observations (ALOS-2 image : 210303)|
|-------------------------------|
|![ALOS-2_1](10fil210303.png)|

| ALOS-2 image (filtered image) | Sentinel-2 image (Mar21) |
|-------------------------------|------------------|
| A |
| ![ALOS-2_A](10fil_1.png) | ![Sentinel-2_A](10fil_2.png) |
| B |
| ![ALOS-2_B](10fil_3.png) | ![Sentinel-2_B](10fil_4.png) |
| C |
| ![ALOS-2_C](10fil_5.png) | ![Sentinel-2_C](10fil_6.png) |

|The observations (ALOS-2 image : 220302)|
|-------------------------------|
|![ALOS-2_1](11fil220302.png)|

| ALOS-2 image (filtered image) | Sentinel-2 image (Mar22) |
|-------------------------------|------------------|
| A |
| ![ALOS-2_A](11fil_1.png) | ![Sentinel-2_A](11fil_2.png) |
| B |
| ![ALOS-2_B](11fil_3.png) | ![Sentinel-2_B](11fil_4.png) |
| C |
| ![ALOS-2_C](11fil_5.png) | ![Sentinel-2_C](11fil_6.png) |

### 3.2.2 Visualize the changes with a color composite and explain the relevance with the production suppression.

Creating a temporal RGB composite image is useful for visualizing differences between multiple SAR images. Since we are working with ALOS-2 images, a common approach is to assign different bands or time-stamped images to the RGB channels.

You can create a RGB composite by combining the two filtered images into the red, green, and blue channels. Since we have two images, one option is to assign one image to the red channel, and the other to the green channel, and use their average or an empty array for the blue channel.

**Steps to create a false-color composite**

1. Use three images or two images and create a synthetic third channel (e.g., by averaging).
2. Assign these to the red, green, and blue channels.
3. Normalize the pixel values so they are in the range [0, 1] to be displayed as an RGB image.
4. Display the false-color composite using mapotlib.

Note: You can replace red_channel, green_channel, and blue_channel with any filtered image paths you prefer, depending on what data or analysis aspect you wish to highlight.

In [ ]:
import re
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Assume filtered_files is a list of file paths
filtered_files = [
    "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-191127_filtered.tif",
    "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-200304_filtered.tif",
    "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-200610_filtered.tif",
    "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-210303_filtered.tif",
    "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-210609_filtered.tif",
    "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-211124_filtered.tif",
    "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-220302_filtered.tif"
]
    
# Extract dates from filenames
def extract_date(filename):
    # Regex pattern to capture dates (e.g., '191127')
    match = re.search(r'\d{6}', filename)
    return match.group(0) if match else "Unknown date"

# Print each file with its index and date
for index, file in enumerate(filtered_files):
    date = extract_date(file)
    print(f"Index {index}: {file} (Date: {date})")

# Specify the indices of the two images to use
red_index = 0  # Image index for the red channel
green_index = 1  # Image index for the green channel
blue_index = 1  # Reuse the green channel image for the blue channel

# Load the images for the specified channels
with rasterio.open(filtered_files[red_index]) as src_red:
    red_channel = src_red.read(1)
with rasterio.open(filtered_files[green_index]) as src_green:
    green_channel = src_green.read(1)
with rasterio.open(filtered_files[blue_index]) as src_blue:
    blue_channel = src_blue.read(1)

# Normalize the channels to the range [0, 1] for RGB visualization
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

# Apply normalization
red_channel_norm = normalize(red_channel)
green_channel_norm = normalize(green_channel)
blue_channel_norm = normalize(blue_channel)

# Stack the normalized channels to create an RGB composite image
rgb_composite = np.stack((red_channel_norm, green_channel_norm, blue_channel_norm), axis=-1)

# Get dates for the channels
red_date = extract_date(filtered_files[red_index])
green_date = extract_date(filtered_files[green_index])
blue_date = extract_date(filtered_files[blue_index])

# Visualize the RGB composite
plt.figure(figsize=(8, 8))
plt.imshow(rgb_composite)
plt.title(f"RGB Composite Image (Red: {red_date}, Green: {green_date}, Blue: {blue_date})")
plt.axis('off')
plt.show()


**Explanation for visualizing RGB composites using Python**

The visual output produced by the RGB composites highlights temporal changes between two ALOS-2 SAR images captured on 191127 and 200304. In the composite, areas that remain unchanged between the two dates are displayed in grayscale, resulting from the combination of RGB. The red regions indicate areas where objects or structures were present only in 191127, suggesting that they were removed or altered by 200304. Conversely, blue regions represent areas where objects appeared only in the 200304 image, indicating new developments or changes that occurred during this time. This type of visualization effectively highlights changes in infrastructure, land use, or other features over time, using the filtered images for precise change detection.

### 3.2.3 Show a time-series profile for a sampled location/region indicating the preduction suppression in TOYOTA.

This code analyzes temporal changes in ALOS-2 backscatter data across specified areas of interest (AOIs) using a stacked time-series raster image. By masking each AOI’s geometry on the stacked raster data, it calculates the median backscatter value for each time layer. The time layers correspond to ALOS-2 images from various dates for visualization purposes. The results are then plotted, showing backscatter trends over time for each AOI, which allows for tracking changes in reflective surface properties, potentially indicating shifts in surface conditions within the AOIs.

**Composing time-series data of ALOS data to identify changes**

Show a time-series profile for a sampled location/region indicating the preduction suppression in TOYOTA.

Composing a time-series stack involves combining multiple ALOS-2 images from different dates into a single multi-layer image. This way, each layer represents data from a specific date, allowing you to analyze temporal changes.

In [ ]:
import rasterio
import numpy as np
import os

# Path to the directory containing filtered images
filtered_images_path = '/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img'

# Initialize a list to store filtered images in a consistent format
filtered_stack = []

# Collect all filtered images into a stack
for filename in os.listdir(filtered_images_path):
    if filename.endswith('.tif'):  # Adjust this if your images are in a different format
        image_path = os.path.join(filtered_images_path, filename)
        with rasterio.open(image_path) as src:
            filtered_image = src.read(1)  # Read the first band (2D)
            filtered_stack.append(filtered_image)

# Convert the list to a 3D numpy array (height, width, time)
time_series_stack = np.stack(filtered_stack, axis=-1)

# Display basic information about the stack
print("Time-Series Stack Shape (Height, Width, Time):", time_series_stack.shape)
print("Number of Layers (Time Dimension):", time_series_stack.shape[-1])


**Export the Time-Series Stack as a GeoTIFF**

In [ ]:
import rasterio
import os

# Initialize a list to store the paths of ALOS files
alos_files = []

# Collect all filtered images into alos_files
for filename in os.listdir(filtered_images_path):
    if filename.endswith('.tif'):  # Adjust this if your images are in a different format
        alos_files.append(os.path.join(filtered_images_path, filename))

# Define the export path for the output GeoTIFF
output_path = 'alos2_time_series_stack_ngo1.tif'

# Open one of the source images to retrieve metadata
with rasterio.open(alos_files[0]) as src:
    # Copy metadata from an existing file and update the count
    metadata = src.meta.copy()
    metadata.update({
        'count': time_series_stack.shape[-1],  # Set count to number of layers (time dimension)
        'dtype': 'float32'                     # Set data type
    })

# Write the multi-layer time-series stack to a GeoTIFF file
with rasterio.open(output_path, 'w', **metadata) as dst:
    for i in range(time_series_stack.shape[-1]):
        # Write each layer as a separate band
        dst.write(time_series_stack[:, :, i].astype('float32'), i + 1)

print(f"Time-Series Stack successfully exported to {output_path}")


![Stacked layer](1stacked.png)

Stacking ALOS-2 images involves combining multiple SAR images taken over the same area but at different times into a single multi-layered file. Here’s the interpretation of the output of the stacked image in different colors representing the following:

**Black and White color:** These indicates no change occurred throughout the time series for that pixel. 

In this case, black and white tones indicate areas where no significant change has occurred across the time span covered by the stacked imagery. These areas remain stable, suggesting consistent levels of activity or unchanged surface conditions.

**Multiple colors:** These colors indicate the areas where the change occurred. Since the stacked layer contains multiple time layers, the values represent the time layer index when a significant change occurs.

In contrast, areas represented by multiple colors highlight locations where changes have occurred. These changes are detected in the backscatter values, which may signify variations in surface properties, such as increased container volume, shifts in port operations, or alterations in industrial activity. 

In [ ]:
pip install geopandas

In [16]:
import rasterio
import rasterio.features  # Import features explicitly
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

**Defines the paths and parameters needed for processing**
- Specify the file path to the stacked ALOS-2 image and any AOI (Area of Interest) files (You can create your own AOI file or download it from [here](https://owncloud.glodal-inc.net/owncloud/index.php/s/xPdaXgH00OwQKFQ)).
- Define the years corresponding to each band in the stacked image, and create simulated time points

In [17]:
stacked_image_path = '/home/jovyan/shared/Sirinya/alos/ALOS2.1/alos2_time_series_stack_ngo1.tif'
aoi_paths = [
    '/home/jovyan/shared/Sirinya/alos/ALOS2.1/aoi_toyota.gpkg'
]
years = [191127, 200304, 200610, 210303, 210609, 211124, 220302]  # Dates of imagery collection
time_points = np.arange(0, len(years) * 7, 7)  # Time intervals as multiples of 7 days

**Reads the stacked ALOS-2 image data**
- Open the stacked ALOS-2 image and read all bands (layers) into a 3D numpy array, with each band
- representing a different time point in the series.

In [18]:
with rasterio.open(stacked_image_path) as src:
    stacked_data = src.read()  # stacked_data has shape (bands, height, width)

**Processes each AOI to calculate median backscatter values**
- Loop through each AOI file. For each AOI, create a mask based on the AOI geometry to isolate the region of interest.
- Calculate the median backscatter value within the AOI for each time point (band) in the stacked image.

In [19]:
median_values_all = {}  # Dictionary to store median values for each AOI

for aoi_path in aoi_paths:
    # Load the AOI geometry using geopandas
    aoi_gdf = gpd.read_file(aoi_path)
    
    # Initialize an empty mask for the entire AOI, combining all geometries
    combined_mask = np.zeros(stacked_data.shape[1:], dtype=bool)
    for geom in aoi_gdf.geometry:
        # If geometry is valid, rasterize it to match the stacked image resolution
        if geom.is_valid:
            mask = rasterio.features.geometry_mask([geom], transform=src.transform,
                                                   invert=True, out_shape=combined_mask.shape)
            combined_mask = np.logical_or(combined_mask, mask)

    # Compute the median backscatter values for each band (time point)
    median_values = []
    for band in range(stacked_data.shape[0]):
        band_data = stacked_data[band]  # Extract data for the current band
        # Calculate median value for masked region
        median_value = np.median(band_data[combined_mask])
        median_values.append(median_value)

    # Store median values for current AOI
    aoi_name = aoi_path.split('/')[-1].split('.')[0]  # Extract AOI name from path
    median_values_all[aoi_name] = median_values


**Plots the median values over time for each AOI**
- Plot the computed median backscatter values for the AOI over time. The x-axis represents time intervals
- and dates, while the y-axis represents the median backscatter value in dB.

In [ ]:
for aoi_name, median_values in median_values_all.items():
    plt.figure(figsize=(10, 6))
    plt.plot(time_points, median_values, marker='o', label='Median AOI Value')

    plt.title(f'Median Value of Change Area Over Time for {aoi_name}')
    plt.xlabel('Date')
    plt.ylabel('Median Backscatter Value')
    plt.xticks(time_points, labels=years, rotation=45)  # Display original years as labels
    plt.grid()
    plt.legend(title='AOI')
    plt.tight_layout()
    plt.show()

**Overall Trend:**
The graph shows a generally declining trend in Toyota's production metrics, indicated by negative values across all dates, with some fluctuations throughout the period. This downward trend is likely tied to the impact of COVID-19 disruptions and semiconductor shortages, which significantly affected production capacity. Toyota's efforts to recover production have been visible, but the data points suggest continued challenges, especially within domestic operations.

**Time-Series Observations:**
- 191127 to 200304: The graph begins with a decline, possibly due to early pandemic-related disruptions and the initial waves of COVID-19.
- 200304 to 200610: A further decrease indicates deepening production impacts, which align with escalating pandemic issues and possibly initial global chip shortages.
- 200610 to 210303: Slight recovery with an increase, suggesting minor improvements as Toyota aimed to resume regular production, despite ongoing pandemic effects.
- 210303 to 210609: A positive shift hints at production adjustments and recovery attempts during this period, even as COVID-19 disruptions continued intermittently.
- 210609 to 211124: A decrease reflects renewed challenges, likely due to COVID-19 resurgences and supply chain constraints affecting Toyota’s output.
- 211124 to 220302: The final point, suggests incremental recovery, potentially as supply issues begin to stabilize and Toyota's production strategy adapts to ongoing limitations.

**Summary:**

The data points illustrate Toyota’s struggle to stabilize production through COVID-19 and a global chip shortage. Early declines reflect the pandemic’s severe impact, while later dates show minor improvements as Toyota resumed operations amid an easing lockdown environment in China. Temporary suspensions at plants and reduced capacity due to workforce shortages contributed to the observed fluctuations. Despite a promising global demand recovery, Toyota's domestic production faced persistent challenges, underscored by recurring dips in the graph. The production outlook remains cautiously optimistic as Toyota adapts to lingering semiconductor shortages and fluctuating pandemic conditions, with gradual recovery potential in the coming quarters.

## 3.3. Change detection of the port operation in the COVID-19 crisis

- Sample observations for pre-crisis, in-crisis, and post-crisis (if available).
- Calculate differences between two observations:
     - pre-crisis and in-crisis: resession of TOYOTA’s productions
     - in-crisis and post-crisis: recovery of TOYOTA’s productions
- Validate the changes for sampled locations on historical imagereis of Google Earth

### 3.3.1 Sample observations for pre-crisis, in-crisis, and post-crisis

The selected images were referred by the article below to represent key phases in Toyota's operations during the COVID-19 pandemic:

- Pre-crisis (191127): This date, November 27, 2019, serves as a baseline image, capturing conditions before the pandemic's impacts took hold in Japan.
- In-crisis (200610): The image from June 10, 2020, aligns with the height of disruptions, particularly in Japan’s manufacturing and automotive sectors, impacted by COVID-19-related production cuts, supply chain disruptions (like chip shortages), and health adaptations. This timing provides insight into the operational challenges Toyota faced during the crisis.
- Post-crisis (211124): By November 24, 2021, Japan was transitioning towards recovery. Toyota and other manufacturers saw signs of recovery, though intermittent COVID-19 outbreaks and residual component shortages still created challenges. This image reflects Toyota's cautious but gradual movement towards full operational capacity.

Note: 
- [Toyota suspends some Japan factory output over COVID outbreak](https://www.reuters.com/business/autos-transportation/toyota-suspends-some-japan-factory-output-over-covid-outbreak-2022-07-27/)
- [Toyota misses April-June output target but says may be turning corner](https://www.reuters.com/business/autos-transportation/toyotas-april-june-global-vehicle-production-falls-98-short-initial-target-2022-07-28/)
- [Toyota suspends some Japan factory production due to COVID outbreak](https://www.reuters.com/business/autos-transportation/toyota-suspends-some-japan-factory-production-due-covid-outbreak-2022-08-09/)

**Define File Paths for Images**

Define the file paths to the pre_crisis, in_crisis, and post_crisis images, so the program knows where to load them. Replace these placeholders with the actual paths on your system.

In [21]:
# Paths to your ALOS-2 images
pre_crisis_image_path = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-191127_filtered.tif"
in_crisis_image_path = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-200610_filtered.tif"
post_crisis_image_path = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-211124_filtered.tif"

**Define a Function to Read and Normalize an Image**

Purpose of `read_image` function: To open an image, read the data, and normalize it for better visualization.

In [22]:
# Create a function to read and normalize an image
def read_image(image_path):
    with rasterio.open(image_path) as src:
        image_data = src.read(1)  # Read the first band
        image_data = (image_data - image_data.min()) / (image_data.max() - image_data.min())  # Normalize to [0, 1]
    return image_data

**Read and Normalize the Images**



In [23]:
# Read and normalize the images
pre_crisis = read_image(pre_crisis_image_path)
in_crisis = read_image(in_crisis_image_path)
post_crisis = read_image(post_crisis_image_path)

**Visualization the images**

In [ ]:

# Set up the figure for visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot each image with color bar
im1 = axes[0].imshow(pre_crisis, cmap='gray')
axes[0].set_title("Pre_crisis: 191127")
axes[0].axis("off")
fig.colorbar(im1, ax=axes[0], orientation='vertical', fraction=0.046, pad=0.04)

im2 = axes[1].imshow(in_crisis, cmap='gray')
axes[1].set_title("In_crisis: 200610")
axes[1].axis("off")
fig.colorbar(im2, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)

im3 = axes[2].imshow(post_crisis, cmap='gray')
axes[2].set_title("Post_crisis: 211124")
axes[2].axis("off")
fig.colorbar(im3, ax=axes[2], orientation='vertical', fraction=0.046, pad=0.04)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()



### 3.3.2 Calculate differences between two observations

- In this section, we use images in 191127 and 200610 to calculate the differences between pre-crisis and in-crisis of COVID-19.
- For the differences between in-crisis and post-crisis of COVID-19, we use images in 200610 and 211124.

#### A. pre-crisis and in-crisis: resession of TOYOTA’s production

In [25]:
# Paths to your ALOS-2 images
pre_crisis_image_path = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-191127_filtered.tif"
in_crisis_image_path = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-200610_filtered.tif"

In [26]:
# Open the images and read the data
with rasterio.open(pre_crisis_image_path) as src_pre:
    pre_crisis_data = src_pre.read(1)  # Read the first band
    pre_crisis_meta = src_pre.meta  # Store metadata for output
    pre_crisis_crs = src_pre.crs  # Store CRS

with rasterio.open(in_crisis_image_path) as src_in:
    in_crisis_data = src_in.read(1)  # Read the first band

In [27]:
# Read the images
pre_crisis = read_image(pre_crisis_image_path)
in_crisis = read_image(in_crisis_image_path)

**Verify the Images Have the Same Shape**

Ensures that the images have matching dimensions, which is required to perform a pixel-wise difference calculation.

In [28]:
# Ensure both images have the same shape
if pre_crisis_data.shape != in_crisis_data.shape:
    raise ValueError("The two images must have the same dimensions.")

**Calculate the Difference between in and post-crisis**

Subtracts each pixel in the pre_crisis image from the corresponding pixel in the in_crisis image, resulting in a "difference" image that highlights changes between the two.

In [29]:
# Calculate the difference
difference_caseA = in_crisis_data - pre_crisis_data

**Visualization the Difference Image**

In [ ]:

# Set up the figure for visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot pre_crisis Image with color bar
im1 = axes[0].imshow(pre_crisis_data, cmap='gray')
axes[0].set_title("Pre_crisis Image : 191127")
axes[0].axis("off")
fig.colorbar(im1, ax=axes[0], orientation='vertical', fraction=0.046, pad=0.04)

# Plot in_crisis Image with color bar
im2 = axes[1].imshow(in_crisis_data, cmap='gray')
axes[1].set_title("In_crisis Image : 200610")
axes[1].axis("off")
fig.colorbar(im2, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)

# Plot Difference Image with color bar
im3 = axes[2].imshow(difference_caseA, cmap='RdYlGn')
axes[2].set_title("Difference (In - Pre)")
axes[2].axis("off")
fig.colorbar(im3, ax=axes[2], orientation='vertical', fraction=0.046, pad=0.04)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


**Plot Histogram of Difference Image (Before Classification) with Mean and Median**

In [ ]:

# Assuming the following variables are defined:
# pre_crisis_data, in_crisis_data

# Case A: Difference between in_crisis_data and pre_crisis_data
difference_caseA = in_crisis_data - pre_crisis_data

# Calculate mean, median, and standard deviation for case 1
mean_diff_caseA = np.mean(difference_caseA)
median_diff_caseA = np.median(difference_caseA)
std_diff_caseA = np.std(difference_caseA)

# Plot histogram for Case A
plt.figure(figsize=(6, 6))
plt.hist(difference_caseA.ravel(), bins=50, color='blue', alpha=0.7)
plt.axvline(mean_diff_caseA, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_diff_caseA:.2f}')
plt.axvline(median_diff_caseA, color='orange', linestyle='dotted', linewidth=1, label=f'Median: {median_diff_caseA:.2f}')
plt.title("Histogram of Difference Image (In Crisis - Pre Crisis)")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.legend()
plt.show()


**Define Classification Thresholds and Apply Classification**

This code classifies each pixel in an image (difference_caseA) into one of three classes based on its difference from the mean pixel value. By categorizing pixels into classes, we can highlight regions with distinct changes. The classification helps identify areas with no change, positive change, and negative change by comparing each pixel to the mean plus or minus one standard deviation. This approach is useful in monitoring the changes over time.

Each class represents a different level of change relative to the mean:

- **No Change (Class 1)**: Pixels assigned a value of 1 have values within one standard deviation of the mean. These pixels represent areas where changes are minimal, indicating stability or no significant change.Pixels
- **Positive Change (Class 2)**: Pixels with a value of 2 are more than one standard deviation above the mean. These pixels represent areas of positive change, suggesting an increase in signal intensity or some new activity in the area.
- **Negative Change (Class 3)**: Pixels with a value of 3 fall more than one standard deviation below the mean, representing areas of negative change. This could indicate a reduction in activity, removal of objects, or other decreases in signal intensity.

In [32]:
classified_image_caseA = np.zeros_like(difference_caseA)  # Initialize classification array

# Class 1: No Change (within one standard deviation of the mean)
classified_image_caseA[(difference_caseA > mean_diff_caseA - std_diff_caseA) & 
                       (difference_caseA <= mean_diff_caseA + std_diff_caseA)] = 1  

# Class 2: Positive Change (greater than one standard deviation above the mean)
classified_image_caseA[difference_caseA > mean_diff_caseA + std_diff_caseA] = 2  

# Class 3: Negative Change (greater than one standard deviation below the mean)
classified_image_caseA[difference_caseA < mean_diff_caseA - std_diff_caseA] = 3


**Visualize the Classified Image**

In [ ]:
# Display the classified image
plt.imshow(classified_image_caseA, cmap='Spectral')
plt.colorbar(label="Change Classes")
plt.title("Classified Change Detection (A)")
plt.axis("off")
plt.show()

**Plot Histogram of Classified Image with Mean and Median**

In [ ]:
# Calculate mean and median of the classified image (class labels) for Case A
mean_classified_caseA = np.mean(classified_image_caseA)
median_classified_caseA = np.median(classified_image_caseA)

# Plot histogram of the classified image for Case A
plt.figure(figsize=(6, 6))
plt.hist(classified_image_caseA.ravel(), bins=np.arange(1, 5) - 0.5, color='blue', alpha=0.7)
plt.axvline(mean_classified_caseA, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_classified_caseA:.2f}')
plt.axvline(median_classified_caseA, color='orange', linestyle='dotted', linewidth=1, label=f'Median: {median_classified_caseA:.2f}')
plt.title("Histogram of Classified Image (Case A: 3 Classes)")
plt.xlabel("Class Label")
plt.ylabel("Frequency")
plt.xticks([1, 2, 3], ['No Change', 'Positive Change', 'Negative Change'])
plt.legend()
plt.grid(axis='y', alpha=0.75)  # Optional: Add grid lines for better readability
plt.show()


**Save the difference image and classified image as GeoTIFF**

In [ ]:
# Paths to save the images for Case A
difference_output_path_caseA = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/pre_in_difference_image_testA.tif"
classified_output_path_caseA = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/classified_change_detection_pre_in_testA.tif"

# Update metadata for saving as GeoTIFF
pre_crisis_meta.update({"driver": "GTiff", "count": 1, "dtype": "float32"})

# Save the difference image for Case 1
with rasterio.open(difference_output_path_caseA, "w", **pre_crisis_meta) as dst:
    dst.write(difference_caseA, 1)  # Write the difference for Case A to the first band

print(f"Difference image for Case A saved to {difference_output_path_caseA}")

# Update metadata for the classified image (using integer data type for class labels)
classified_meta_caseA = pre_crisis_meta.copy()
classified_meta_caseA.update({"dtype": "int32"})

# Save the classified image for Case A
with rasterio.open(classified_output_path_caseA, "w", **classified_meta_caseA) as dst:
    dst.write(classified_image_caseA, 1)  # Write the classified image for Case A to the first band

print(f"Classified Change Detection image for Case A saved to {classified_output_path_caseA}")


#### B: in-crisis and post-crisis: recovery of TOYOTA’s productions

In [36]:
# Paths to your ALOS-2 images
in_crisis_image_path = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-200610_filtered.tif"
post_crisis_image_path = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/filtered_img/HH-211124_filtered.tif"

In [37]:
# Open the images and read the data
with rasterio.open(in_crisis_image_path) as src_pre:
    in_crisis_data = src_pre.read(1)  # Read the first band
    in_crisis_meta = src_pre.meta  # Store metadata for output
    in_crisis_crs = src_pre.crs  # Store CRS

with rasterio.open(post_crisis_image_path) as src_post:
    post_crisis_data = src_post.read(1)  # Read the first band

In [38]:
# Read the images
in_crisis = read_image(in_crisis_image_path)
post_crisis = read_image(post_crisis_image_path)

In [39]:
# Ensure both images have the same shape
if in_crisis_data.shape != post_crisis_data.shape:
    raise ValueError("The two images must have the same dimensions.")

**Calculate the Difference**

Subtracts each pixel in the in_crisis image from the corresponding pixel in the post_crisis image, resulting in a "difference" image that highlights changes between the two.

In [40]:
# Calculate the difference
difference_caseB = post_crisis_data - in_crisis_data

**Visualization the Difference Image**

In [ ]:

# Set up the figure for visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot in_crisis Image with color bar
im1 = axes[0].imshow(in_crisis_data, cmap='gray')
axes[0].set_title("In_crisis Image : 200610")
axes[0].axis("off")
fig.colorbar(im1, ax=axes[0], orientation='vertical', fraction=0.046, pad=0.04)

# Plot post_crisis Image with color bar
im2 = axes[1].imshow(post_crisis_data, cmap='gray')
axes[1].set_title("Post_crisis Image : 211124")
axes[1].axis("off")
fig.colorbar(im2, ax=axes[1], orientation='vertical', fraction=0.046, pad=0.04)

# Plot Difference Image with color bar
im3 = axes[2].imshow(difference_caseB, cmap='RdYlGn')
axes[2].set_title("Difference (Post - In)")
axes[2].axis("off")
fig.colorbar(im3, ax=axes[2], orientation='vertical', fraction=0.046, pad=0.04)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


**Plot Histogram of Difference Image (Before Classification) with Mean and Median**

In [ ]:
# Assuming the following variable is defined:
# post_crisis_data

# Case B: Difference between post_crisis_data and in_crisis_data
difference_caseB = post_crisis_data - in_crisis_data

# Calculate mean, median, and standard deviation for case B
mean_diff_caseB = np.mean(difference_caseB)
median_diff_caseB = np.median(difference_caseB)
std_diff_caseB = np.std(difference_caseB)

# Plot histogram for Case B
plt.figure(figsize=(6, 6))
plt.hist(difference_caseB.ravel(), bins=50, color='green', alpha=0.7)
plt.axvline(mean_diff_caseB, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_diff_caseB:.2f}')
plt.axvline(median_diff_caseB, color='orange', linestyle='dotted', linewidth=1, label=f'Median: {median_diff_caseB:.2f}')
plt.title("Histogram of Difference Image (Post Crisis - In Crisis)")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.legend()
plt.show()



**Define Classification Thresholds and Apply Classification**

This code classifies each pixel in an image (difference_caseA) into one of three classes based on its difference from the mean pixel value. By categorizing pixels into classes, we can highlight regions with distinct changes. The classification helps identify areas with no change, positive change, and negative change by comparing each pixel to the mean plus or minus one standard deviation. This approach is useful in monitoring the changes over time.

Each class represents a different level of change relative to the mean:

- **No Change (Class 1)**: Pixels assigned a value of 1 have values within one standard deviation of the mean. These pixels represent areas where changes are minimal, indicating stability or no significant change.Pixels
- **Positive Change (Class 2)**: Pixels with a value of 2 are more than one standard deviation above the mean. These pixels represent areas of positive change, suggesting an increase in signal intensity or some new activity in the area.
- **Negative Change (Class 3)**: Pixels with a value of 3 fall more than one standard deviation below the mean, representing areas of negative change. This could indicate a reduction in activity, removal of objects, or other decreases in signal intensity.

In [43]:
classified_image_caseB = np.zeros_like(difference_caseB)  # Initialize classification array

# Class 1: No Change (within one standard deviation of the mean)
classified_image_caseB[(difference_caseB > mean_diff_caseB - std_diff_caseB) & 
                       (difference_caseB <= mean_diff_caseB + std_diff_caseB)] = 1  

# Class 2: Positive Change (greater than one standard deviation above the mean)
classified_image_caseB[difference_caseB > mean_diff_caseB + std_diff_caseB] = 2  

# Class 3: Negative Change (greater than one standard deviation below the mean)
classified_image_caseB[difference_caseB < mean_diff_caseB - std_diff_caseB] = 3

**Visualize the Classified Image**

In [ ]:
# Display the classified image
plt.imshow(classified_image_caseB, cmap='Spectral')
plt.colorbar(label="Change Classes")
plt.title("Classified Change Detection (B)")
plt.axis("off")
plt.show()


**Plot Histogram of Classified Image with Mean and Median**

In [ ]:
# Calculate mean and median of the classified image (class labels) for Case B
mean_classified_caseB = np.mean(classified_image_caseB)
median_classified_caseB = np.median(classified_image_caseB)

# Plot histogram of the classified image for Case B
plt.figure(figsize=(6, 6))
plt.hist(classified_image_caseB.ravel(), bins=np.arange(1, 5) - 0.5, color='green', alpha=0.7)
plt.axvline(mean_classified_caseB, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_classified_caseB:.2f}')
plt.axvline(median_classified_caseB, color='orange', linestyle='dotted', linewidth=1, label=f'Median: {median_classified_caseB:.2f}')
plt.title("Histogram of Classified Image (Case B: 3 Classes)")
plt.xlabel("Class Label")
plt.ylabel("Frequency")
plt.xticks([1, 2, 3], ['No Change', 'Positive Change', 'Negative Change'])
plt.legend()
plt.grid(axis='y', alpha=0.75)  # Optional: Add grid lines for better readability
plt.show()

**Save the difference image and classified image as GeoTIFF**

In [ ]:
# Paths to save the images for Case B
difference_output_path_caseB = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/in_post_difference_image_testB.tif"
classified_output_path_caseB = "/home/jovyan/shared/Sirinya/alos/ALOS2.1/Shimpomachi_ngo/classified_change_detection_in_post_testB.tif"

# Update metadata for saving as GeoTIFF
in_crisis_meta.update({"driver": "GTiff", "count": 1, "dtype": "float32"})

# Save the difference image for Case B
with rasterio.open(difference_output_path_caseB, "w", **in_crisis_meta) as dst:
    dst.write(difference_caseB, 1)  # Write the difference for Case B to the first band

print(f"Difference image for Case B saved to {difference_output_path_caseB}")

# Update metadata for the classified image (using integer data type for class labels)
classified_meta_caseB = in_crisis_meta.copy()
classified_meta_caseB.update({"dtype": "int32"})

# Save the classified image for Case 2
with rasterio.open(classified_output_path_caseB, "w", **classified_meta_caseB) as dst:
    dst.write(classified_image_caseB, 1)  # Write the classified image for Case B to the first band

print(f"Classified Change Detection image for Case B saved to {classified_output_path_caseB}")



### 3.3.3 Validate the changes for sampled locations on historical imagereis of Google Earth

**How to validate the changes for sampled locations on historical imageries of Sentinel-2**

- Using differences between pre-typhoon and post-typhoon image.
- Using the filtered data to validate changed area by using pre-typhoon (October 3, 2019) and post-typhoon (November 3, 2019) image.
- Check the historical imagereis in Google Earth. Ig the Google Earth has no image in the day/period that we want to check, we can download Landsat or Sentinel-2 to check the historical imagereis instead.
- In this case, we downloaded the Sentinel-2 images from [Google Earth Engine](https://code.earthengine.google.com/3ce320461a9fa2c7402b8e5f2f793f7f).
- Download [QGIS](http://test.qgis.org/html/en/site/forusers/download.html) software.
- Open differences between pre-typhoon and post-typhoon image, filtered data in pre-typhoon and post-typhoon image, and Sentinel-2 data in pre-typhoon and post-typhoon image in QGIS to visual validate changed area.

**Pre-crisys and In-crisis:**

**Sample location 1**

|Difference between Pre-crisis (191127) and In-crisis (200610)|
|-------------------------------|
|![12Dif_pi1](12Dif_pi1.png)|

|ALOS-2 (filtered): Pre-crisis (191127) | Sentinel-2: Pre-crisis (Nov19)|
|-------------------------------|------------------|
| ![12Dif_pi1_1](12Dif_pi1_1.png) | ![12Dif_pi1_2](12Dif_pi1_2.png) |

|ALOS-2 (filtered): Pre-crisis (200610) | Sentinel-2: In-crisis (Jun20)|
|-------------------------------|------------------|
| ![12Dif_pi1_3](12Dif_pi1_3.png) | ![12Dif_pi1_4](12Dif_pi1_4.png) |

**Sample location 2**

|Difference between Pre-crisis (191127) and In-crisis (200610)|
|-------------------------------|
|![12Dif_pi2](12Dif_pi2.png)|

|ALOS-2 (filtered): Pre-crisis (191127) | Sentinel-2: Pre-crisis (Nov19)|
|-------------------------------|------------------|
| ![12Dif_pi2_1](12Dif_pi2_1.png) | ![12Dif_pi2_2](12Dif_pi2_2.png) |

|ALOS-2 (filtered): Pre-crisis (200610) | Sentinel-2: In-crisis (Jun20)|
|-------------------------------|------------------|
| ![12Dif_pi2_3](12Dif_pi2_3.png) | ![12Dif_pi2_4](12Dif_pi2_4.png) |

**In-crisis and Post-crisis:**

**Sample location 1**

|Difference between In-crisis (200610) and Post-crisis (211124)|
|-------------------------------|
|![13Dif_ip1](13Dif_ip1.png)|

|ALOS-2 (filtered): In-crisis (200610) | Sentinel-2: In-crisis (Jun20)|
|-------------------------------|------------------|
| ![13Dif_ip1_1](13Dif_ip1_1.png) | ![13Dif_ip1_2](13Dif_ip1_2.png) |

|ALOS-2 (filtered): Post-crisis (211124) | Sentinel-2: Post-crisis (Nov21)|
|-------------------------------|------------------|
| ![13Dif_ip1_3](13Dif_ip1_3.png) | ![13Dif_ip1_4](13Dif_ip1_4.png) |

**Sample location 2**

|Difference between In-crisis (200610) and Post-crisis (211124)|
|-------------------------------|
|![13Dif_ip2](13Dif_ip2.png)|

|ALOS-2 (filtered): In-crisis (200610) | Sentinel-2: In-crisis (Jun20)|
|-------------------------------|------------------|
| ![13Dif_ip2_1](13Dif_ip2_1.png) | ![13Dif_ip2_2](13Dif_ip2_2.png) |

|ALOS-2 (filtered): Post-crisis (211124) | Sentinel-2: Post-crisis (Nov21)|
|-------------------------------|------------------|
| ![13Dif_ip2_3](13Dif_ip2_3.png) | ![13Dif_ip2_4](13Dif_ip2_4.png) |